## cuda 확인, accelerate install

In [ ]:
from transformers import pipeline

# Ensure that CUDA is available
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA A100-SXM4-40GB


In [ ]:
torch.cuda.empty_cache()  # GPU 캐시 메모리 정리
import gc
gc.collect()  # 가비지 컬렉터 실행

118

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

# Test 1
## 모델 명 : [M-SOLAR-10.7B-v1.3 ](https://huggingface.co/megastudyedu/M-SOLAR-10.7B-v1.3)
## 10.7B + BF16 => OOM 발생

### 성능 테스트 코드 1 (뉴스 나눠서 요약)

In [ ]:
# 뉴스 데이터
news_data = [
    {
        "topic": "증권",
        "title": "메타 주가 단기 고점 찍었나…저커버그, 주식 5억弗 처분",
        "link": "https://n.news.naver.com/mnews/article/015/0004933064",
        "content": "두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 \"상승 랠리 끝나간다\"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.\n\n\n\n마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.\n\n\n\n외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fed)이 올해부터 금리를 내릴 것이라고 밝혔지만, 최근 금리 인하폭이 시장 예상만큼 크지 않을 것이란 전망이 고개를 들고 있다. 경기 연착륙 확률을 낮춰 잡는 투자자가 늘어나면 미국 주식을 사는 수요도 줄어들 수 있다.",
        "summary": "이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. 마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 ..",
        "press": "한국경제",
        "journalist": "박의명 기자",
        "date": "2024-01-04 18:19:01"
    },
    {
        "topic": "증권",
        "title": "새해 몸값 1조 대어급 IPO 몰려온다…'탄환'은 충분",
        "link": "https://n.news.naver.com/mnews/article/374/0000365465",
        "content": "[앵커] 최근 주식시장에서는 '공모주 투자는 불패'라는 말이 나올 정도로 공모주 인기가 상당한데요. 새해에도 몸값이 조 단위를 넘나 드는 대어급 공모주가 속속 등장하고 있어 관심을 끌고 있습니다. 김동필 기자, 올해 초부터 상당히 큰 규모의 기업공개(IPO)가 예정돼 있네요? [기자] 배우 김희선 씨와 유재석 씨를 광고 모델로 내세워 잘 알려진 뷰티 테크 기업 에이피알이 다음 달 코스피 상장을 목표로 준비 중인데요. 상장 후 시가총액은 2조 원에 가까울 것으로 보여 '대어'로 꼽힙니다. 이외에도 HD현대마린솔루션이나 토스 운영사 비바리퍼블리카, 플랜텍 등도 출격 대기 중입니다. [앵커] 연초부터 관심을 모으는 건 아무래도 작년부터 이어진 공모주 열풍 영향이 크죠? [기자] 특히 작년 말 상장 첫날 주가가 4배 상승한 종목들이 잇따르면서 공모주가 돈이 된다는 '불패' 이미지까지 생겼는데요. 투자자들 사이에선 '묻지마 청약' 얘기도 나오는 상황입니다. 공모주로 향할 수 있는 투자자 예탁금도 60조 원에 육박해 대기자금은 충분합니다. 다만 왜곡 우려도 적진 않습니다. [이경준 / 혁신IB자산운용 대표 : 무조건 수익을 낸다라고 해서 다 들어갔다가 결국에 떨어지면 피해는 결국 나중에 들어오는 투자자에게 가기 때문에… 묻지마 투자의 최고봉이 아닌가. 지금 시장이. 그래서 건전한 자본시장은 확실하게 아니다.] 은행 예금금리가 낮아지고 있어 새해 IPO 기대감이 커지고 있습니다. SBS Biz 김동필입니다.당신의 제보가 뉴스로 만들어집니다.SBS Biz는 여러분의 제보를 기다리고 있습니다.홈페이지 = https://url.kr/9pghjn",
        "summary": "최근 주식시장에서는 '공모주 투자는 불패'라는 말이 나올 정도로 공모주 인기가 상당한데요. 새해에도 몸값이 조 단위를 넘나 드는 대어급..",
        "press": "SBS Biz",
        "journalist": "김동필 기자",
        "date": "2024-01-04 18:13:04"
    },
    {
        "topic": "증권",
        "title": "\"어느새 반토막\"…이차전지주 '빌빌'",
        "link": "https://n.news.naver.com/mnews/article/277/0005363135",
        "content": "전기차 시장 성장 둔화 영향4분기 실적 부진 전망 '우려'이차전지주들의 주가가 지난해 고점 대비 절반 수준으로 하락하는 등 좀처럼 반등하지 못하고 있다. 전기차 등 이차전지 전방시장 둔화 우려가 지속해서 영향을 끼치는 것으로 보인다.\n\n\n\n4일 LG에너지솔루션의 종가는 41만9000원이다. 종가 기준 지난해 6월12일 기록한 사상 최고가 61만2000원 대비 31.54% 하락했다. 삼성SDI도 마찬가지다. 79만3000원에서 44만5000원으로 밀리며 43.88% 떨어졌다. 이는 코스닥에 상장된 이차전지주들도 마찬가지다. 에코프로와 에코프로비엠은 지난해 고점이었던 129만3000원과 46만2000원에서 63만4000원과 27만9000원으로 빠졌다. 고점 대비 각각 50.97%, 39.61% 하락했다.이차전지주들의 하락은 전방시장인 전기차 수요 둔화 현상이 영향을 끼친 것으로 풀이된다. 에너지 전문 시장조사업체 SNE리서치에 따르면 작년 전 세계 전기차 등록 대수는 약 1377만대를 기록할 전망이다. 이는 지난해 상반기 예측한 1484만대 대비 107만대 줄어든 수치다. 성장률 역시 36.4%에서 30.6%로 하향 조정하는 등 전기차 시장이 둔화할 것으로 예상했다.실제 글로벌 전기차 판매량은 둔화하고 있다. 삼성증권에 따르면 작년 11월 글로벌 친환경차 판매량은 142만대로 전년 동월 대비 29% 늘었다. 하지만 전월 대비로는 7.24% 증가에 그쳤다.전기차 시장 둔화의 가장 큰 이유는 소비 위축이다. 고금리로 사람들이 자동차를 구매하기 어려워진 상태다. 여기에 전기차의 일반 내연기관 자동차 대비 비싸다. 또 국가별로 다르지만, 전기차 보조금을 축소시키거나 폐지시키고 있는 곳이 늘어나고 있다.전방시장의 둔화 우려가 커지면서 이차전지 업체들의 실적도 악화할 것으로 예상된다. 이안나 유안타증권 연구원은 \"국내 이차전지 섹터는 전기차 배터리 판매량 감소 및 평균판매단가(ASP) 하락과 일회성 비용 등으로 저조한 4분기 실적이 예상된다\"며 \"양극재 기업들은 3분기에 이어 4분기에도 셀 기업들의 재고소진이 이어지면서 저조한 가동률이 지속하고 있는 데다 일회성 비용까지 더해져 영업이익 적자가 예상된다\"고 말했다.장정훈 삼성증권 연구원도 \"이달 말부터 셀 업체를 시작으로 지난해 4분기 성과를 점검하게 될 것으로 보인다\"며 \"지난해 3분기 실적 발표 시점인 10월 말 상황과 비교하면 4분기 중후반 유럽 및 미국 완성차 업체들의 판매 부진으로 시장 예상치를 하회할 가능성이 높아 보인다\"고 설명했다.",
        "summary": "이차전지주들의 주가가 지난해 고점 대비 절반 수준으로 하락하는 등 좀처럼 반등하지 못하고 있다. 전기차 등 이차전지 전방시장 둔화 우려..",
        "press": "아시아경제",
        "journalist": "유현석 기자",
        "date": "2024-01-04 18:00:00"
    }

]

### 1) 질문 넣지 않고 pipleline의 "summarization" 그대로 활용 예시

In [ ]:
from transformers import pipeline
import torch

# GPU 사용 설정
device = 0 if torch.cuda.is_available() else -1

# Summarization pipeline 초기화
# pipe = pipeline(task="summarization", model="logicker/SkkuDataScience-10.7B-v5", device=accelerator.device)
# pipe = pipeline(task="summarization", model="JaeyeonKang/CCK-v1.0.0-DPO", device=accelerator.device)
pipe = pipeline(task="summarization", model="megastudy/M-SOLAR-10.7B-v1.3", device=accelerator.device)


# 각 뉴스 기사를 요약
#summaries = [pipe(article["content"]) for article in news_data]

summaries = []
for article in news_data:
    # 텍스트 길이 줄이기 (예: 앞에서부터 512단어만 사용)
    content = ' '.join(article["content"].split()[:512])

    # 요약 실행
    with accelerator:

      summary = pipe(content)
    summaries.append(summary[0]['summary_text'])

    # GPU 메모리 캐시 비우기
    torch.cuda.empty_cache()

# 요약된 결과 출력
for summary in summaries:
    print(summary[0]['summary_text'])


config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/9.83G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacty of 39.56 GiB of which 30.81 MiB is free. Process 19920 has 39.53 GiB memory in use. Of the allocated memory 39.10 GiB is allocated by PyTorch, and 16.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### 질문 넣고 , pipeline의 text-generation 사용

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import pipeline
import torch

# GPU 사용 설정
device = 0 if torch.cuda.is_available() else -1

# 텍스트 생성 파이프라인 초기화
pipe = pipeline("text-generation", model="megastudy/M-SOLAR-10.7B-v1.3", device=device)

def ask(question, context, max_new_tokens=512, temperature=0.7, top_p=0.9):
    # 질문과 맥락을 함께 입력으로 사용
    input_text = f"### 질문: {question}\n\n### 맥락: {context}\n\n### 답변:"

    # 텍스트 생성 실행
    ans = pipe(input_text, do_sample=True, max_new_tokens=max_new_tokens, temperature=temperature, top_p=top_p, return_full_text=False)
    return ans[0]['generated_text']


# 각 뉴스 기사에 대한 질문 및 답변 생성
for article in news_data:
    question = "이 기사의 주요 내용은 무엇인가요?"
    context = article['content']
    answer = ask(question, context)
    print(f"기사 제목: {article['title']}")
    print(f"답변: {answer}\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacty of 39.56 GiB of which 116.81 MiB is free. Process 27448 has 39.44 GiB memory in use. Of the allocated memory 39.02 GiB is allocated by PyTorch, and 16.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## 성능 테스트 코드 2 (뉴스 한번에 요약)

*   함수로 질문, 뉴스 데이터 한번에 넣음



In [ ]:
# GPU 사용 설정
device = 0 if torch.cuda.is_available() else -1
print(device)
#pipe = pipeline(task="summarization", model="logicker/SkkuDataScience-10.7B-v5", device=0)

# 위에꺼 돌리고 돌릴땐 실행안해도됨
#pipe = pipeline("text-generation", model="megastudy/M-SOLAR-10.7B-v1.3", device=device)


def ask(question, context='', max_new_tokens=512, temperature=0.7, top_p=0.9):
    # 질문과 맥락을 함께 입력으로 사용
    input_text = f"### 질문: {question}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {question}\n\n### 답변:"

    # 텍스트 생성 실행
    ans = pipe(input_text, do_sample=True, max_new_tokens=max_new_tokens, temperature=temperature, top_p=top_p, return_full_text=False)
    print(ans[0]['generated_text'])


# 뉴스 데이터를 하나의 텍스트로 결합
input_text = ' '.join([article['content'] for article in news_data])

# 질문 실행
instruction = "오늘 주식 상황을 크게 3가지로 요약해줘"
ask(instruction, input_text)

### 위와 같은 코드인데, pipline 말고 autotockenizer, automodelforcausalLM 사용

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# GPU 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("megastudy/M-SOLAR-10.7B-v1.3")
model = AutoModelForCausalLM.from_pretrained("megastudy/M-SOLAR-10.7B-v1.3").to(device)


In [ ]:
def ask(question, context='', max_new_tokens=512, temperature=0.7, top_p=0.9):
    # 질문과 맥락을 결합하여 입력 텍스트 생성
    input_text = f"### 질문: {question}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {question}\n\n### 답변:"
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # 텍스트 생성
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True
    )

    # 생성된 텍스트 디코딩
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(answer)


input_text = ' '.join([article['content'] for article in news_data])

In [ ]:
# 질문 실행
instruction = "오늘 주식 상황을 크게 3가지로 요약해줘"
ask(instruction, input_text)

# 경량화 (bitsandbytes)
- 모델의 Embedding 레이어(단어를 수치 벡터로 변환하는 부분)에 대해 이러한 경량화를 적용
- 모델의 다른 부분(예: 히든 레이어, 출력 레이어)에는 영향을 주지 않으며, 주로 메모리 사용량이 많은 Embedding 레이어에 초점을 맞춤

 ### ex) 32비트 대 8비트 경량화:
 - **일반적인 32비트 저장**:   
대부분의 딥러닝 모델에서는 파라미터(가중치)를 32비트 부동소수점(Floating Point, FP32) 형태로 저장함.     
ex) 하나의 파라미터가 0.123456789라는 값을 가진다고 할 때, 이 값은 32비트 메모리 공간에 저장됨.

 - **8비트 경량화**:   
 bitsandbytes를 사용하여 이러한 파라미터를 8비트 정밀도로 줄이면, 같은 값을 대략적으로 근사하여 더 적은 메모리에 저장함.    
ex) 0.123456789는 8비트 공간에 0.123 또는 비슷한 값으로 근사 저장될 수 있음. 이는 원래 값보다 정밀도는 낮지만, 메모리 사용량을 크게 줄일 수 있음.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# 8 bit 경량화

## hyeogi/SOLAR-10.7B-dpo-v1
### 10.9B + FP16 + 8bit
- https://huggingface.co/hyeogi/SOLAR-10.7B-dpo-v1
- Model size : 10.9B params
- Tensor type : FP16

In [2]:
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("hyeogi/SOLAR-10.7B-dpo-v1")
model = AutoModelForCausalLM.from_pretrained("hyeogi/SOLAR-10.7B-dpo-v1", load_in_8bit=True, device_map="cuda:0")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
import torch
torch.cuda.empty_cache()  # GPU 캐시 메모리 정리
import gc
gc.collect()  # 가비지 컬렉터 실행

158

In [ ]:
# 질문 실행
instruction = "오늘 주식 상황에 대해서 요약해줘봐"
ask(instruction, input_text)

### 질문: 오늘 주식 상황에 대해서 요약해줘봐

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙

In [ ]:
# 질문 실행
instruction = "이차전지 관련 소식 있어?"
ask(instruction, input_text)

### 질문: 이차전지 관련 소식 있어?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fe

In [ ]:
# 질문 실행
instruction = "메타 관련 호재가 있어?"
ask(instruction, input_text)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


### 질문: 메타 관련 호재가 있어?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fed

In [ ]:
# 질문 실행
instruction = "메타 관련 호재가 있어?"
ask(instruction, input_text)

### 질문: 메타 관련 호재가 있어?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fed

In [ ]:
instruction = "뉴스 기사 요약해줘"
ask(instruction, input_text)

### 질문: 뉴스 기사 요약해줘

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fed)이 

# 4bit 경량화

## megastudy/M-SOLAR-10.7B-v1.3
### 10.7B + BF16 + 4bit 경량화 (성공)

- https://huggingface.co/megastudy/M-SOLAR-10.7B-v1.3
- Model size : 10.7B params
- Tensor type : BF16


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", load_in_4bit=True, device_map="auto")

model = AutoModelForCausalLM.from_pretrained("megastudy/M-SOLAR-10.7B-v1.3", load_in_4bit=True, device_map="cuda:0")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# GPU 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("megastudy/M-SOLAR-10.7B-v1.3")
#model = AutoModelForCausalLM.from_pretrained("megastudy/M-SOLAR-10.7B-v1.3").to(device)

def ask(question, context='', max_new_tokens=512, temperature=0.7, top_p=0.9):
    # 질문과 맥락을 결합하여 입력 텍스트 생성
    input_text = f"### 질문: {question}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {question}\n\n### 답변:"
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # 텍스트 생성
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True
    )

    # 생성된 텍스트 디코딩
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(answer)


input_text = ' '.join([article['content'] for article in news_data])

# 질문 실행
instruction = "오늘 주식 상황을 크게 3가지로 요약해줘"
ask(instruction, input_text)


cuda


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


### 질문: 오늘 주식 상황을 크게 3가지로 요약해줘

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국

In [ ]:
instruction = "다음 주식 뉴스들을 요약해줘"
ask(instruction, input_text)

### 질문: 다음 주식 뉴스들을 요약해줘

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(F

In [ ]:
instruction = "메타와 관련된 주식 뉴스 있어?"
ask(instruction, input_text)

### 질문: 메타와 관련된 주식 뉴스 있어?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행

#### 같은 모델 8bit 양자화

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("megastudy/M-SOLAR-10.7B-v1.3")
model = AutoModelForCausalLM.from_pretrained("megastudy/M-SOLAR-10.7B-v1.3", load_in_8bit=True, device_map="cuda:0")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
instruction = "오늘 주식 상황을 크게 3가지로 요약해줘"
ask(instruction, input_text)

### 질문: 오늘 주식 상황을 크게 3가지로 요약해줘

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국

In [ ]:
instruction = "다음 주식 뉴스들을 요약해줘"
ask(instruction, input_text)

### 질문: 다음 주식 뉴스들을 요약해줘

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(F

In [ ]:
instruction = "메타와 관련된 주식 뉴스 있어?"
ask(instruction, input_text)

### 질문: 메타와 관련된 주식 뉴스 있어?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행

In [ ]:
instruction = "이차전지 관련 소식있으면 알려줘"
ask(instruction, input_text)

### 질문: 이차전지 관련 소식있으면 알려줘

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행

In [ ]:
instruction = "이차전지 주식 살까 말까 ?"
ask(instruction, input_text)

### 질문: 이차전지 주식 살까 말까 ?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(F

# 작은 모델 Test

## zomd/AISquare-Instruct-yi-ko-6b-v0.9.30 (작은 모델 )

- https://huggingface.co/zomd/AISquare-Instruct-yi-ko-6b-v0.9.30
- Model size : 6.18B params
- Tensor type : BF16
- Base Model [beomi/Yi-Ko-6B](https://huggingface.co/beomi/Yi-Ko-6B)
- Yi-Ko series models are an auto-regressive language model that uses an optimized transformer architecture based on ***Llama-2****.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("zomd/AISquare-Instruct-yi-ko-6b-v0.9.30", device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained("zomd/AISquare-Instruct-yi-ko-6b-v0.9.30")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### 새로운 모델 load 후 아래 코드 실행하면 됨

In [ ]:
# 뉴스 데이터
news_data = [
    {
        "topic": "증권",
        "title": "메타 주가 단기 고점 찍었나…저커버그, 주식 5억弗 처분",
        "link": "https://n.news.naver.com/mnews/article/015/0004933064",
        "content": "두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 \"상승 랠리 끝나간다\"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.\n\n\n\n마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.\n\n\n\n외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fed)이 올해부터 금리를 내릴 것이라고 밝혔지만, 최근 금리 인하폭이 시장 예상만큼 크지 않을 것이란 전망이 고개를 들고 있다. 경기 연착륙 확률을 낮춰 잡는 투자자가 늘어나면 미국 주식을 사는 수요도 줄어들 수 있다.",
        "summary": "이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. 마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 ..",
        "press": "한국경제",
        "journalist": "박의명 기자",
        "date": "2024-01-04 18:19:01"
    },
    {
        "topic": "증권",
        "title": "새해 몸값 1조 대어급 IPO 몰려온다…'탄환'은 충분",
        "link": "https://n.news.naver.com/mnews/article/374/0000365465",
        "content": "[앵커] 최근 주식시장에서는 '공모주 투자는 불패'라는 말이 나올 정도로 공모주 인기가 상당한데요. 새해에도 몸값이 조 단위를 넘나 드는 대어급 공모주가 속속 등장하고 있어 관심을 끌고 있습니다. 김동필 기자, 올해 초부터 상당히 큰 규모의 기업공개(IPO)가 예정돼 있네요? [기자] 배우 김희선 씨와 유재석 씨를 광고 모델로 내세워 잘 알려진 뷰티 테크 기업 에이피알이 다음 달 코스피 상장을 목표로 준비 중인데요. 상장 후 시가총액은 2조 원에 가까울 것으로 보여 '대어'로 꼽힙니다. 이외에도 HD현대마린솔루션이나 토스 운영사 비바리퍼블리카, 플랜텍 등도 출격 대기 중입니다. [앵커] 연초부터 관심을 모으는 건 아무래도 작년부터 이어진 공모주 열풍 영향이 크죠? [기자] 특히 작년 말 상장 첫날 주가가 4배 상승한 종목들이 잇따르면서 공모주가 돈이 된다는 '불패' 이미지까지 생겼는데요. 투자자들 사이에선 '묻지마 청약' 얘기도 나오는 상황입니다. 공모주로 향할 수 있는 투자자 예탁금도 60조 원에 육박해 대기자금은 충분합니다. 다만 왜곡 우려도 적진 않습니다. [이경준 / 혁신IB자산운용 대표 : 무조건 수익을 낸다라고 해서 다 들어갔다가 결국에 떨어지면 피해는 결국 나중에 들어오는 투자자에게 가기 때문에… 묻지마 투자의 최고봉이 아닌가. 지금 시장이. 그래서 건전한 자본시장은 확실하게 아니다.] 은행 예금금리가 낮아지고 있어 새해 IPO 기대감이 커지고 있습니다. SBS Biz 김동필입니다.당신의 제보가 뉴스로 만들어집니다.SBS Biz는 여러분의 제보를 기다리고 있습니다.홈페이지 = https://url.kr/9pghjn",
        "summary": "최근 주식시장에서는 '공모주 투자는 불패'라는 말이 나올 정도로 공모주 인기가 상당한데요. 새해에도 몸값이 조 단위를 넘나 드는 대어급..",
        "press": "SBS Biz",
        "journalist": "김동필 기자",
        "date": "2024-01-04 18:13:04"
    },
    {
        "topic": "증권",
        "title": "\"어느새 반토막\"…이차전지주 '빌빌'",
        "link": "https://n.news.naver.com/mnews/article/277/0005363135",
        "content": "전기차 시장 성장 둔화 영향4분기 실적 부진 전망 '우려'이차전지주들의 주가가 지난해 고점 대비 절반 수준으로 하락하는 등 좀처럼 반등하지 못하고 있다. 전기차 등 이차전지 전방시장 둔화 우려가 지속해서 영향을 끼치는 것으로 보인다.\n\n\n\n4일 LG에너지솔루션의 종가는 41만9000원이다. 종가 기준 지난해 6월12일 기록한 사상 최고가 61만2000원 대비 31.54% 하락했다. 삼성SDI도 마찬가지다. 79만3000원에서 44만5000원으로 밀리며 43.88% 떨어졌다. 이는 코스닥에 상장된 이차전지주들도 마찬가지다. 에코프로와 에코프로비엠은 지난해 고점이었던 129만3000원과 46만2000원에서 63만4000원과 27만9000원으로 빠졌다. 고점 대비 각각 50.97%, 39.61% 하락했다.이차전지주들의 하락은 전방시장인 전기차 수요 둔화 현상이 영향을 끼친 것으로 풀이된다. 에너지 전문 시장조사업체 SNE리서치에 따르면 작년 전 세계 전기차 등록 대수는 약 1377만대를 기록할 전망이다. 이는 지난해 상반기 예측한 1484만대 대비 107만대 줄어든 수치다. 성장률 역시 36.4%에서 30.6%로 하향 조정하는 등 전기차 시장이 둔화할 것으로 예상했다.실제 글로벌 전기차 판매량은 둔화하고 있다. 삼성증권에 따르면 작년 11월 글로벌 친환경차 판매량은 142만대로 전년 동월 대비 29% 늘었다. 하지만 전월 대비로는 7.24% 증가에 그쳤다.전기차 시장 둔화의 가장 큰 이유는 소비 위축이다. 고금리로 사람들이 자동차를 구매하기 어려워진 상태다. 여기에 전기차의 일반 내연기관 자동차 대비 비싸다. 또 국가별로 다르지만, 전기차 보조금을 축소시키거나 폐지시키고 있는 곳이 늘어나고 있다.전방시장의 둔화 우려가 커지면서 이차전지 업체들의 실적도 악화할 것으로 예상된다. 이안나 유안타증권 연구원은 \"국내 이차전지 섹터는 전기차 배터리 판매량 감소 및 평균판매단가(ASP) 하락과 일회성 비용 등으로 저조한 4분기 실적이 예상된다\"며 \"양극재 기업들은 3분기에 이어 4분기에도 셀 기업들의 재고소진이 이어지면서 저조한 가동률이 지속하고 있는 데다 일회성 비용까지 더해져 영업이익 적자가 예상된다\"고 말했다.장정훈 삼성증권 연구원도 \"이달 말부터 셀 업체를 시작으로 지난해 4분기 성과를 점검하게 될 것으로 보인다\"며 \"지난해 3분기 실적 발표 시점인 10월 말 상황과 비교하면 4분기 중후반 유럽 및 미국 완성차 업체들의 판매 부진으로 시장 예상치를 하회할 가능성이 높아 보인다\"고 설명했다.",
        "summary": "이차전지주들의 주가가 지난해 고점 대비 절반 수준으로 하락하는 등 좀처럼 반등하지 못하고 있다. 전기차 등 이차전지 전방시장 둔화 우려..",
        "press": "아시아경제",
        "journalist": "유현석 기자",
        "date": "2024-01-04 18:00:00"
    }

]


input_text = ' '.join([article['content'] for article in news_data])



def ask(question, context='', max_new_tokens=512, temperature=0.7, top_p=0.9):
    # 질문과 맥락을 결합하여 입력 텍스트 생성
    input_text = f"### 질문: {question}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {question}\n\n### 답변:"
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device="cuda:0")

    # 텍스트 생성
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True
    )

    # 생성된 텍스트 디코딩
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(answer)


In [ ]:
# 질문 실행
instruction = "오늘 주식 상황을 크게 3가지로 요약해줘"
ask(instruction, input_text)

 ### 질문: 오늘 주식 상황을 크게 3가지로 요약해줘

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미

In [ ]:
instruction = "오늘 주식 상황이 어때 ?"
ask(instruction, input_text)

 ### 질문: 오늘 주식 상황이 어때 ?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(F

In [ ]:
instruction = "메타 관련 뉴스 있어 ?"
ask(instruction, input_text)

 ### 질문: 메타 관련 뉴스 있어 ?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fe

In [ ]:
instruction = "이차전지 관련 뉴스 봤을때 매수하는게 좋을것같아 매도하는게 좋을것 같아 ?"
ask(instruction, input_text)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


 ### 질문: 이차전지 관련 뉴스 봤을때 매수하는게 좋을것같아 매도하는게 좋을것 같아 ?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미

In [ ]:
instruction = "너가 대답할수있는 답변글자는 몇이야 ?"
ask(instruction, input_text)

 ### 질문: 너가 대답할수있는 답변글자는 몇이야 ?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국

In [ ]:
instruction = "메타 살까 말까?"
ask(instruction, input_text)

 ### 질문: 메타 살까 말까?

### 맥락: 두 달 동안 128만여주 매도메타 등 美 빅테크 주가 조정 증권가 "상승 랠리 끝나간다"이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다.



마크 저커버그 메타플랫폼스 최고경영자(CEO·사진)가 5600억원 규모의 메타 주식을 처분했다. 저커버그가 보유 주식을 매도한 것은 2021년 11월 이후 처음이다. 월가에선 빅테크 주식들이 고점을 친 것 아니냐는 관측도 나오고 있다.4일 블룸버그에 따르면 저커버그 CEO는 지난해 11월 1일부터 12월 말까지 128만여 주를 매도해 4억2800만달러(약 5600억원)를 현금화했다. 이날 시간외거래에서 메타는 약 0.01% 오른 344달러에 거래됐다. 대량 매도에도 주가가 급락하지 않은 이유는 거래가 사전에 정한 수량, 날짜에 이뤄졌기 때문이다. 미국에서 기업 내부자가 자사 주식을 매매하려면 120일 전에 미국 증권거래위원회(SEC)에 신고해야 한다.



외신에 따르면 올해 39세인 저커버그는 메타 지분 약 13%를 보유하고 있다. 순자산은 1250억달러(약 163조원)로 추정된다.시장에선 메타 주가가 상승세를 이어갈지 여부를 놓고 갑론을박이 벌어지고 있다. 메타 주가는 지난 1년간 170% 넘게 올랐다. 3일(현지시간) 종가는 344.47달러로 2022년 11월 저점(90.79달러)과 비교하면 약 네 배 높은 수준이다.메타를 포함한 미국 빅테크 주식들은 지난해 말부터 조정을 받고 있다. 애플, 아마존, 알파벳, 마이크로소프트, 메타, 테슬라, 엔비디아의 주가를 추종하는 ‘블룸버그 매그니피센트7 지수’는 4거래일 연속 하락(-3.4%)했다.전문가들은 이미 주가가 충분히 오른 데다 인공지능(AI) 시장 확대에 대한 기대가 줄어들면서 차익실현 물량이 나오고 있다고 분석했다. 스티브 소스닉 인터랙티브브로커스 전략가는 “모멘텀이 다시 나타나지 않는다면 상승 랠리가 끝날 것”이라고 말했다.미국의 통화정책도 변수로 꼽힌다. 미국 중앙은행(Fed)이 